# Библиотеки

In [1]:
from tqdm.notebook import tqdm
import numpy as np
import json
import torch
from nltk.tokenize import RegexpTokenizer
from sklearn.metrics import classification_report
from torch.utils.tensorboard import SummaryWriter
import re
import string

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
!git clone https://github.com/kbayazitov/VK_nlp

Cloning into 'VK_nlp'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 12 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), done.


# Выборка

In [5]:
texts = []
for line in open('/content/VK_nlp/data.jsonl', 'r'):
    texts.append(json.loads(line))

In [6]:
len(texts)

1300

In [7]:
texts[0][:5]

[['С 1768 года Россия ведёт войну против Османской империи.', 1],
 ['В середине января 1770 года корпус генерала Христофора Штофельна, призванный очистить Молдавию и Валахию от турок, разместился в столице Молдавского княжества городе Яссы.',
  0],
 ['Там русскую армию встретил враг, по жестокости не уступающий башибузукам – чума.',
  0],
 ['За зиму она унесла жизни нескольких тысяч солдат.', 0],
 ['Вскоре эпидемия вспыхнула в близлежащей Польше, а в августе достигла Киева: к началу ноября 1770 года из 20-ти тысяч жителей города умерло более шести.',
  1]]

# Описание идеи

Эта задача очень схожа с задачей распознавания именнованных сущностей, которая хорошо решается с помощью LSTM модели. Однако в данном случае у нас вместо разметки слов - разметка предложений. Нужно придумать метод получения векторного представления предложений. Опять же напрашивается LSTM модель (Так как снова работаем с последовательностью - слова в предложении)

Будем использовать композицию из двух biLSTM моделей. Первая модель $SentenceLSTM$ будет принимать предложения в виде эмбеддингов слов и возвращать эмбеддинг всего предложения на основе макспулинга. Вторая модель $TextLSTM$ будет принимать эмбеддинги предложений из одного текста и возвращать метки классов после полносвязного слоя. 

# Import Fasttext

Для векторизации текстов будем использовать сжатую fast-text модель с $dim=300$

In [8]:
!pip install -q compress-fasttext

     |████████████████████████████████| 24.1 MB 1.3 MB/s 


In [9]:
import compress_fasttext

In [10]:
ft = compress_fasttext.models.CompressedFastTextKeyedVectors.load(
    'https://github.com/avidale/compress-fasttext/releases/download/gensim-4-draft/geowac_tokens_sg_300_5_2020-100K-20K-100.bin'
)

# Модель

In [11]:
class SentenceLSTM(torch.nn.Module):
    @property
    def device(self):
        return next(self.parameters()).device

    def __init__(self, emb_dim, hidden_dim):
        super(SentenceLSTM, self).__init__()
        self.encoder = torch.nn.LSTM(emb_dim, hidden_dim, num_layers=2, 
                                     bidirectional=True, 
                                     batch_first=True, dropout=0.1)
        self.emb_dim = emb_dim
        self.hidden_dim = hidden_dim
        
    def forward(self, input):
        batch_size = input.shape[0]
        act, _ = self.encoder(input)
        res = torch.zeros(batch_size, act.shape[2])
        for i in range(batch_size):
            res[i, :] = torch.max(act[i], 0)[0] # макспулинг
        return res

In [12]:
class TextLSTM(torch.nn.Module):
    @property
    def device(self):
        return next(self.parameters()).device

    def __init__(self, sentence_encoder, hidden_dim, output_dim):
        super(TextLSTM, self).__init__()
        self.encoder = torch.nn.LSTM(sentence_encoder.hidden_dim * 2,
                                     hidden_dim, num_layers=2, 
                                     bidirectional=True, 
                                     batch_first=True, dropout=0.1)

        self.linear = torch.nn.Linear(2*hidden_dim, output_dim)        
        self.sentence_encoder = sentence_encoder

    def forward(self, input):
        encoded_sentence = self.sentence_encoder(input).to(device)
        act, _ = self.encoder(encoded_sentence)
        return self.linear(act)

# Код для обучения 

In [13]:
def trainer(count_of_epoch, 
            dataset,
            model, 
            loss_function,
            optimizer,
            lr = 0.001):

    optima = optimizer(model.parameters(), lr=lr)
    
    iterations = tqdm(range(count_of_epoch), desc='epoch')

    for it in iterations:
        for ds in dataset:
            batch_generator = torch.utils.data.DataLoader(dataset=ds, batch_size=len(ds))
            for it, (x_batch, y_batch) in enumerate(batch_generator):
                model.train()
                optima.zero_grad()

                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)

                output = model(x_batch)
    
                loss = loss_function(output, y_batch)
                loss.backward()
                optima.step()

# Токенизатор + Векторизатор 

Сначала токенизируем тексты с помощью CustomTokenizer, далее векторизуем на основе FastText

In [14]:
class CustomTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        text = re.sub(f'[{string.punctuation}]', ' ', text) 
        t = RegexpTokenizer(r'[a-zа-яёЁА-ЯA-Z]+|[^\w\s]|\d+')
        tokens = t.tokenize(text.lower())
        return tokens

In [15]:
class VectorizerFastText(object):
    def __init__(self, ft, tokenizer):
        self.ft = ft
        self.tokenizer = tokenizer
    def __call__(self, sentences, max_length=None):
        
        if max_length is None:
            max_length = max(len(sentences[i]) for i in range(len(sentences)))

        res = []
        for sent in sentences:
            tokens = self.tokenizer.tokenize(sent)

            if len(tokens) < max_length:
                tokens = tokens + ['[PAD]']*(max_length-len(tokens))
            else:
                tokens = tokens[:max_length]

            res.append(tokens)

        vectors = np.array([[self.ft[w] for w in sent] for sent in res])
        return torch.tensor(vectors, dtype=torch.float32)

In [16]:
Tokenizer = CustomTokenizer()
Vectorizer = VectorizerFastText(ft, Tokenizer)

# Эксперимент

### Создание выборки

In [17]:
dataset = []
for text in tqdm(texts):
    sents = list(list(zip(*text))[0])
    tags = list(list(zip(*text))[1])
    vect_sents = Vectorizer(sents, max_length=25) # Оставим в каждом предложении только первые 25 токенов (Больше не позволяет ОЗУ)
    vect_text = torch.utils.data.TensorDataset(vect_sents, torch.tensor(tags).long())
    dataset.append(vect_text)

dataset_train_pt = dataset[:int(len(dataset)*0.8)]
dataset_test_pt = dataset[int(len(dataset)*0.2):]

  0%|          | 0/1300 [00:00<?, ?it/s]

### Инициализация модели + Обучение

In [ ]:
'''loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam

config = dict()
config['sentence_encoder'] = SentenceLSTM(emb_dim=300, hidden_dim=100).to(device)
config['hidden_dim'] = 100
config['output_dim'] = 3

model = TextLSTM(**config)
model.to(device)

trainer(count_of_epoch=50,  
        dataset=dataset_train_pt,
        model=model, 
        loss_function=loss_function,
        optimizer=optimizer,
        lr=0.001)
'''

In [19]:
model = torch.load('/content/VK_nlp/model.pt')

### Результаты

In [20]:
pred = []
real = []

for ds in dataset_test_pt:
    batch_generator = torch.utils.data.DataLoader(dataset=ds, batch_size=len(ds))
    model.eval()
    for it, (x_batch, y_batch) in enumerate(batch_generator):
        x_batch = x_batch.to(device)
        output = model(x_batch)

    pred.extend(torch.argmax(output, dim=-1).cpu().numpy().tolist())
    real.extend(y_batch.cpu().numpy().tolist())

print(classification_report(real, pred))

              precision    recall  f1-score   support

           0       0.92      0.91      0.91     62158
           1       0.77      0.80      0.79     26732
           2       0.70      0.61      0.65      3039

    accuracy                           0.87     91929
   macro avg       0.80      0.77      0.78     91929
weighted avg       0.87      0.87      0.87     91929



# Автоматическое разбиение текста

In [21]:
# Функция, разбивающия текст. В случае sections=True - разбиение на секции, иначе на абзацы
def split_text(text, sections=True):
    if sections == True:
        temp = 2 # Переменная отвечающая за метку класса по которому идет разбиение
    else:
        temp = 1
    # Разбиение текста на предложения
    sentences = text.split('.')
    if sentences[len(sentences)-1] == '':
        sentences = sentences[:len(sentences)-1]
    # Векторизация предложений
    vect_text = Vectorizer(sentences, max_length=25).to(device)
    res = model(vect_text)
    labels = torch.argmax(res, dim=-1).cpu().numpy().tolist()
    # Получим индексы предложений-начал секций
    beginnings_of_segments = []
    for i in range(len(labels)):
        if labels[i] == temp:
            beginnings_of_segments.append(i)
    # Заводим лист секций
    segments = [[] for i in range(len(beginnings_of_segments))]
    i = -1
    # Заполняем лист
    for j, label in enumerate(labels):
        if label == temp:
            i += 1
        segments[i].append(sentences[j])

    # Джойним предложения внутри каждого сегмента
    res = []
    for segment in segments:
        res.append('.'.join(segment))

    # Вывод результата
    for i, r in enumerate(res):
        if sections == True:
            print(f'Section {i}: {r}')
        else:
            print(f'Paragraph {i}: {r}')

In [22]:
test_text = ' '.join(list(list(zip(*texts[-14]))[0]))

In [23]:
split_text(test_text, sections=True)

Section 0:  Выходу перевода «Властелина колец» препятствовала государственная цензура, поскольку философия Толкина, по мнению советских цензоров, не соответствовала линии партии. Основная претензия заключалась в том, что «Властелин Колец» содержит «пессимистическую концепцию о необратимом влиянии зла на историческое развитие», которая не укладывалась в оптимистические рамки соцреализма, господствовавшего в советской литературе. Толкин призывал читателя «быть на высоте положения» и отвечать на вызовы Зла, не теряя надежды на торжество Добра, «если оно ещё сильно». В Советском Союзе такую идею, очевидно, сочли содержащей политический подтекст, из-за чего книге не давали зелёный свет более 15 лет. Но в узких кругах московской и ленинградской интеллигенции о «Властелине Колец» знали не понаслышке ещё с 1960-х годов, когда появились первые самиздатовские переводы произведений Толкина. Энтузиасты на коленке переводили «идеологически невыдержанные» романы «буржуазного писателя», переписывали 

In [24]:
split_text(test_text, sections=False)

Paragraph 0: Споры о лучшем варианте перевода «Властелина Колец» ведутся с начала 1990-х годов, когда советскому, а затем и российскому читателю стали доступны на выбор около десятка разнообразных переводов произведения Джона Толкина. Одни упирали на «высокохудожественность» трудов Кистяковского и Муравьёва, другим нравилась «точность» Грузберга, а третьи предпочитали «академичность» Каррика и Каменкович с многостраничными комментариями и пояснениями толкиновского языка. В дискуссиях о достоинствах и недостатках каждого перевода читатели, однако, забывают о людях, которые годами пытались адаптировать текст Толкина под особенности русского языка
Paragraph 1:  TJ подробно рассказывает о переводчиках-первопроходцах, каждый из которых независимо друг от друга переводил «Властелина колец» сначала в 1960-е, потом в 1970-е, а затем в 1980-е годы
Paragraph 2:  Официально «Властелин Колец» появился в СССР лишь в 1982 году — спустя 28 лет после первой публикации трёхтомника в Великобритании. Тог

# Вывод

Получили неплохое качество классификации. Одним из главных минусов данной работы в том что используются только первые 25 токенов каждого предложения.